In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the data
data = pd.read_csv('/Users/macbook/Documents/PhD_Documents/sentiment_methods/sentiments/results/sentiment_data.csv')

# Drop the unnamed column if it exists
data = data.drop('Unnamed: 0', axis=1)

# Convert the 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Sort the DataFrame by the 'Date' column
data = data.sort_values(by='Date')

# Display the sorted DataFrame
print(data)

           Date   finbert  textblob     vader  crudebert  count
0    2014-01-01 -0.162637 -0.125000 -0.192800  -0.071561      6
1    2014-01-02 -0.033916 -0.079048 -0.068061  -0.326042     77
2    2014-01-03 -0.089735 -0.043608 -0.065303  -0.327837     72
3    2014-01-04 -0.156314 -0.466667 -0.529933  -0.292674      3
4    2014-01-05 -0.267569 -0.166116 -0.324973  -0.426660     11
...         ...       ...       ...       ...        ...    ...
3678 2024-01-27 -0.002451  0.026892 -0.005786  -0.073699     63
3679 2024-01-28 -0.043316 -0.020916 -0.103613  -0.272669     87
3680 2024-01-29 -0.061280 -0.048330 -0.094058  -0.242793    244
3681 2024-01-30 -0.045728 -0.041854 -0.056878  -0.396974    295
3682 2024-01-31 -0.057176 -0.057087 -0.097403  -0.422442    383

[3683 rows x 6 columns]


In [3]:
# Load data from CSV file
df = pd.read_csv('/Users/macbook/Documents/PhD_Documents/ECAI/Financial_data/cb-5m.csv',
                 delimiter=';', names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])

# Convert 'Date' and 'Time' columns to datetime format
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S')
df['Date'] = df['Datetime'].dt.date  # Keep only the date part

# Calculate returns (log returns)
df['Return'] = np.log(df['Close'] / df['Close'].shift(1))

# Scale returns by 100
df['Return'] *= 100

# Calculate realized volatility (RV)
df['RV'] = df['Return']**2
realized_volatility = df.groupby('Date')['RV'].sum().reset_index()
realized_volatility.columns = ['Date', 'RV']

# Calculate bipower variation (BVP)
df['Abs_Return'] = df['Return'].abs()
df['BVP'] = df['Abs_Return'].shift(1) * df['Abs_Return']
bipower_variation = df.groupby('Date')['BVP'].sum().reset_index()
bipower_variation.columns = ['Date', 'BVP']

# Calculate negative and positive realized volatility (RV_neg, RV_pos)
df['RV_neg'] = np.where(df['Return'] < 0, df['Return']**2, 0)
df['RV_pos'] = np.where(df['Return'] > 0, df['Return']**2, 0)

neg_realized_volatility = df.groupby('Date')['RV_neg'].sum().reset_index()
neg_realized_volatility.columns = ['Date', 'RV_neg']

pos_realized_volatility = df.groupby('Date')['RV_pos'].sum().reset_index()
pos_realized_volatility.columns = ['Date', 'RV_pos']

# Calculate realized quadratic variation (RQ)
df['RQ'] = df['Return']**4
realized_quadratic_variation = df.groupby('Date')['RQ'].sum().reset_index()
realized_quadratic_variation.columns = ['Date', 'RQ']

# Merge all the calculations into one DataFrame
result = pd.merge(realized_volatility, bipower_variation, on='Date')
result = pd.merge(result, neg_realized_volatility, on='Date')
result = pd.merge(result, pos_realized_volatility, on='Date')
result = pd.merge(result, realized_quadratic_variation, on='Date')

# Ensure the Date column is in the format y-m-d
result['Date'] = pd.to_datetime(result['Date']).dt.strftime('%Y-%m-%d')

# Print or save the result
print(result)

            Date         RV        BVP     RV_neg     RV_pos          RQ
0     2009-01-26   0.519063   0.332713   0.067739   0.451324    0.100434
1     2009-01-27  23.483746  14.588539  15.041501   8.442245   16.925468
2     2009-01-28  26.454882  16.344508  13.320336  13.134546   20.546776
3     2009-01-29  17.664604  11.170477   8.144389   9.520215    6.320081
4     2009-01-30  13.757901   8.287309   7.254396   6.503505    5.045175
...          ...        ...        ...        ...        ...         ...
4896  2024-10-24   2.964736   1.612153   1.764732   1.200003    0.218300
4897  2024-10-25   1.996323   1.031267   0.736433   1.259891    0.206959
4898  2024-10-27  25.211006   0.841348  24.639343   0.571663  590.863654
4899  2024-10-28   3.186080   2.012130   1.697866   1.488214    0.211401
4900  2024-10-29   2.635110   1.533892   1.419651   1.215458    0.158226

[4901 rows x 6 columns]


In [4]:
result['Date'] = pd.to_datetime(result['Date'])  # 'errors=coerce' will turn invalid parsing into NaT
data['Date'] = pd.to_datetime(data['Date'])

In [5]:
merged_df = pd.merge(data, result, on='Date', how='inner')
merged_df

Date   finbert  textblob     vader  crudebert  count        RV  \
0    2014-01-01 -0.162637 -0.125000 -0.192800  -0.071561      6  0.038377   
1    2014-01-02 -0.033916 -0.079048 -0.068061  -0.326042     77  1.310692   
2    2014-01-03 -0.089735 -0.043608 -0.065303  -0.327837     72  0.722298   
3    2014-01-05 -0.267569 -0.166116 -0.324973  -0.426660     11  0.085836   
4    2014-01-06 -0.094459 -0.064420 -0.089700  -0.291635     94  0.576129   
...         ...       ...       ...       ...        ...    ...       ...   
3130 2024-01-26 -0.033161 -0.021683 -0.049067  -0.360818    297  2.541310   
3131 2024-01-28 -0.043316 -0.020916 -0.103613  -0.272669     87  2.160446   
3132 2024-01-29 -0.061280 -0.048330 -0.094058  -0.242793    244  3.273172   
3133 2024-01-30 -0.045728 -0.041854 -0.056878  -0.396974    295  2.579344   
3134 2024-01-31 -0.057176 -0.057087 -0.097403  -0.422442    383  4.178235   

           BVP    RV_neg    RV_pos        RQ  
0     0.016711  0.008113  0.030264  0.000222  
1     0.736862  0.999830  0.310862  0.067245  
2     0.434809  0.427129  0.295169  0.010593  
3     0.053793  0.044708  0.041128  0.000749  
4     0.324593  0.317377  0.258751  0.008342  
...        ...       ...       ...       ...  
3130  1.573123  1.126665  1.414645  0.209726  
3131  0.750572  0.593345  1.567101  1.669870  
3132  1.841890  2.028347  1.244824  0.284698  
3133  1.490120  1.199505  1.379839  0.124942  
3134  2.745514  2.554867  1.623368  0.444401  

[3135 rows x 11 columns]

In [6]:
merged_df.to_csv('sentiment_Brent.csv')